In [7]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [8]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

In [9]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

## XGBoost With PCA and With Feature Engineering

In [23]:
files = ['MeanWeeklyImputed', 'KNNWeeklyImputed', 'MovingAverageWeeklyImputed', 'RegressionWeeklyImputed', 'MIIWeeklyImputed']

for file in files:
    df = read_csv(file+'_PCA_training_FE.csv')
    print(file)
    X_test = read_csv(file + '_PCA_test_FE.csv')

    y = pd.read_csv('sp500_target_regimes.csv', parse_dates=True)
    y.index = y['date'].values
    y = y[['regime']]
    y = y['regime']=='BEAR'
    y = pd.DataFrame (y, columns = ['regime'])
       
    
    X_train = df
    y_train = y.loc['2000-01-01':'2015-01-01']
    y_test = y.loc['2015-01-02':]
    
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    param_test1 = {
        'max_depth':range(2,10,1),
        'min_child_weight':range(1,10,1),
        'subsample':np.arange(0.5,1,0.1),
        #'gamma':np.arange(0.1,0.5,0.1),
        #'max_delta_step':range(1,10,1),
        }
    
    balance = len(y_train[y_train['regime']==False])/len(y_train[y_train['regime']==True])
    
    gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(objective="binary:logistic",
                                                          #disable_default_eval_metric = 1,
                                                          learning_rate=0.1,
                                                          n_estimators=500, 
                                                          random_state=27, 
                                                          min_child_weight=2,
                                                          subsample=0.1,
                                                          #gamma=3, 
                                                          eval_metric=['auc', 'error'],
                                                          max_depth=2, 
                                                          #max_delta_step=1,
                                                          #scale_pos_weight=balance
                                                         ), 
                            param_grid = param_test1, 
                            scoring='f1',
                            n_jobs=4,
                            iid=False, 
                            cv=5)

    gsearch1.fit(X_train, y_train, )#eval_metric=f1_eval)
    print(gsearch1.best_params_, gsearch1.best_score_)
    
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                                  #disable_default_eval_metric = 1,
                                  learning_rate=0.1, 
                                  n_estimators=500, 
                                  random_state=27,
                                  eval_metric=['auc', 'error'],
                                  #max_delta_step=gsearch1.best_params_['max_delta_step'],
                                  min_child_weight=gsearch1.best_params_['min_child_weight'],
                                  #gamma=gsearch1.best_params_['gamma'], 
                                  max_depth=gsearch1.best_params_['max_depth'],  
                                  #scale_pos_weight=balance,
                                  subsample=gsearch1.best_params_['subsample'],  
                                 )

    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=100)#,eval_metric=f1_eval)
    y_train_pred = xgb_model.predict(X_train, ntree_limit=xgb_model.best_ntree_limit)
    print("_______________________________________")
    print("______________Training_________________")
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")
    print("_______________Testing_________________")
    y_pred = xgb_model.predict(X_test, ntree_limit=xgb_model.best_ntree_limit)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")


MeanWeeklyImputed
(782, 92) (782, 1) (272, 92) (272, 1)
{'max_depth': 2, 'min_child_weight': 4, 'subsample': 0.5} 0.7518767322273601
_______________________________________
______________Training_________________
     True False
True    366    15
False   6    395
_______________________________________
F1 score 0.02788844621513953
---------------------------------------
_______________Testing_________________
     True False
True    50    58
False   15    149
_______________________________________
F1 score 0.4219653179190751
---------------------------------------
KNNWeeklyImputed
(782, 80) (782, 1) (272, 80) (272, 1)
{'max_depth': 2, 'min_child_weight': 8, 'subsample': 0.5} 0.7554888517524405
_______________________________________
______________Training_________________
     True False
True    240    28
False   132    382
_______________________________________
F1 score 0.25
---------------------------------------
_______________Testing_________________
     True False
True    31   